In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
data_folder1 = './BeerAdvocate/'
data_folder2 = './matched_beer_data/'

The purpose of the following analysis was to observe where the beers that each user rates are brewed and which beer styles are prefered by each user depending on the nationality of the user. 

In [3]:
df_beers = pd.read_csv(data_folder1 + 'beers.csv')
df_breweries = pd.read_csv(data_folder1 + 'breweries.csv')
df_users = pd.read_csv(data_folder1 + 'users.csv')
df_matched = pd.read_csv(data_folder2 + 'ratings.csv')

/Users/lolavegeas/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,2,3,5,7,8,9,10,11,13,17,18,19,20,22,24,25,26,27,29,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_ba = df_matched.iloc[:,:17]
df_ba.columns = df_ba.iloc[0,:].values
df_ba = df_ba.drop(0).reset_index(drop=True)

In [5]:
df_ba.head()

,abv,appearance,aroma,beer_id,beer_name,brewery_id,brewery_name,date,overall,palate,rating,review,style,taste,text,user_id,user_name
0,11.3,4.5,4.5,645,Trappistes Rochefort 10,207,Brasserie de Rochefort,1324810800,5.0,4.5,4.8,True,Quadrupel (Quad),5.0,Best before 27.07.2016Directly reviewed in com...,erzengel.248045,Erzengel
1,5.0,NaN,NaN,28191,Myanmar Lager Beer,9369,Myanmar Brewery and Distillery,1322650800,NaN,NaN,3.0,True,American Adjunct Lager,NaN,NaN,visionthing.639993,visionthing
2,5.0,3.5,3.5,57911,Cantillon Tyrnilambic Baie D’Argousier Lambic,388,Brasserie Cantillon,1344074400,4.0,4.0,3.85,True,Lambic - Fruit,4.0,"Bottle @ One Pint Pub, Helsinki. 2006 vintage....",tiong.608427,tiong
3,5.0,4.0,3.5,57913,Cantillon Pikkulinnun Viskilambic,388,Brasserie Cantillon,1344074400,4.0,4.0,3.68,True,Lambic - Unblended,3.5,"Originally rated on 16.11.2009, draught @ Pikk...",tiong.608427,tiong
4,6.0,4.0,4.0,81125,Drie Fonteinen Oude Geuze - Armand & Tommy,2216,Brouwerij 3 Fonteinen,1346234400,4.0,4.0,4.0,True,Gueuze,4.0,"750ml bottle, originally rated on 18.8.2012.Bo...",tiong.608427,tiong


In [6]:
df_breweries.columns = ['brewery_id','location','brewery_name','nbr_beers']

In [7]:
countries = ["United States","Canada","Utah","England","Wales","Scotland"]

# merge the df_ba dataset with the df_breweries dataset to get the location where the beers are brewed
df_ba_merged = df_ba.merge(df_breweries[['brewery_id','location','nbr_beers']], left_on='brewery_id', right_on='brewery_id')

# convert dates to datetime
df_ba_merged["date"] = df_ba_merged["date"].apply(pd.to_datetime,unit='s')

# creates column with corresponding country where  beer is brewed
df_ba_merged["country"] = df_ba_merged["location"]
for country in countries:
    df_ba_merged["bool"] = df_ba_merged["location"].apply(lambda x: country in x)
    if country == "Utah":
        country = "United States"
    df_ba_merged["country"].loc[df_ba_merged["bool"]] = country
    del df_ba_merged['bool']
    
df_ba_merged.head()

/Users/lolavegeas/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,abv,appearance,aroma,beer_id,beer_name,brewery_id,brewery_name,date,overall,palate,rating,review,style,taste,text,user_id,user_name,location,nbr_beers,country
0,6.3,3.5,3.0,46146,Tallgrass IPA,16333,Tallgrass Brewing Company,2011-04-09 10:00:00,3.0,3.5,3.08,True,American IPA,3.0,"16 ounce can, 11 232 stamped on the bottom. Se...",bluejacket74.27421,bluejacket74,"United States, Kansas",80,United States
1,5.2,4.0,4.0,78937,Tallgrass 8-Bit Pale Ale,16333,Tallgrass Brewing Company,2012-03-16 11:00:00,4.0,3.5,3.95,True,American Pale Ale (APA),4.0,16oz. can served in a SA perfect pint glass. P...,homebrewhawk.34031,homebrewhawk,"United States, Kansas",80,United States
2,6.8,NaN,NaN,93034,Ethos IPA,16333,Tallgrass Brewing Company,2014-12-15 11:00:00,NaN,NaN,4.0,True,American IPA,NaN,NaN,devman171.732781,Devman171,"United States, Kansas",80,United States
3,5.2,NaN,NaN,78937,Tallgrass 8-Bit Pale Ale,16333,Tallgrass Brewing Company,2014-12-15 11:00:00,NaN,NaN,3.75,True,American Pale Ale (APA),NaN,NaN,devman171.732781,Devman171,"United States, Kansas",80,United States
4,5.2,NaN,NaN,78937,Tallgrass 8-Bit Pale Ale,16333,Tallgrass Brewing Company,2014-07-12 10:00:00,NaN,NaN,3.75,True,American Pale Ale (APA),NaN,NaN,tectactoe.666880,tectactoe,"United States, Kansas",80,United States


In [8]:
# creates column with corresponding user country
df_users["user_country"] = df_users["location"]
for country in countries:
    df_users["bool"] = df_users[df_users["location"].notnull()]["location"].apply(lambda x: country in x)
    if country == "Utah":
        country = "United States"
    df_users["user_country"].loc[df_users["bool"] == True] = country
    del df_users['bool']

/Users/lolavegeas/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [9]:
df_users.columns = ["user_nbr_ratings", "user_nbr_reviews", "user_id", "user_name", "date_joined", "location", "user_country"]

In [10]:
# merge df_ba_merged with df_users to get country of origin of user
df_ba_user = df_ba_merged.merge(df_users[['user_nbr_ratings','user_nbr_reviews','user_id','user_country']], left_on='user_id', right_on='user_id')
df_ba_user.head()

,abv,appearance,aroma,beer_id,beer_name,brewery_id,brewery_name,date,overall,palate,...,taste,text,user_id,user_name,location,nbr_beers,country,user_nbr_ratings,user_nbr_reviews,user_country
0,6.3,3.5,3.0,46146,Tallgrass IPA,16333,Tallgrass Brewing Company,2011-04-09 10:00:00,3.0,3.5,...,3.0,"16 ounce can, 11 232 stamped on the bottom. Se...",bluejacket74.27421,bluejacket74,"United States, Kansas",80,United States,2766,2766,United States
1,6.0,3.0,3.25,132406,King Gose Home Imperial Gose,14879,Hoppin' Frog Brewery,2014-12-30 11:00:00,3.25,3.5,...,3.25,"22 ounce bottle. Served in a Unibroue tulip, ...",bluejacket74.27421,bluejacket74,"United States, Ohio",104,United States,2766,2766,United States
2,10.5,4.5,4.0,69407,Sixteen,392,Weyerbacher Brewing Co.,2011-07-23 10:00:00,4.0,4.0,...,4.0,"12 ounce bottle, bottled on 04/14/2011. Served...",bluejacket74.27421,bluejacket74,"United States, Pennsylvania",184,United States,2766,2766,United States
3,7.7,4.25,4.0,98561,Althea,392,Weyerbacher Brewing Co.,2014-03-01 11:00:00,4.0,3.75,...,4.0,"750 ml bottle. Served in a goblet, the beer po...",bluejacket74.27421,bluejacket74,"United States, Pennsylvania",184,United States,2766,2766,United States
4,10.2,3.5,3.5,37692,Twelve,392,Weyerbacher Brewing Co.,2011-04-01 10:00:00,3.5,3.5,...,3.5,"22 ounce bottle. Served in a snifter, the beer...",bluejacket74.27421,bluejacket74,"United States, Pennsylvania",184,United States,2766,2766,United States


In [11]:
# create dictionary with beers rated for each user
df_ba_grouped_by_user = df_ba_user.groupby("user_id")
users_dict = {}
for key,group in df_ba_user.groupby("user_id"):
    users_dict[key] = pd.DataFrame(columns=['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'date','rating','style','abv','user','country','user_nbr_ratings','user_nbr_reviews','user_country'])
    group_sorted = group.sort_values(by="date")
    users_dict[key] = group_sorted[['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'date','rating', 'style','abv', 'user_name', 'country','user_nbr_ratings','user_nbr_reviews','user_country']]

In [12]:
# example for one user
users_dict['badbeer.25091'].head()

,beer_id,beer_name,brewery_id,brewery_name,date,rating,style,abv,user_name,country,user_nbr_ratings,user_nbr_reviews,user_country
2364,779,Weihenstephaner Korbinian,252,Bayerische Staatsbrauerei Weihenstephan,2005-12-18 11:00:00,4.1,Doppelbock,7.4,badbeer,Germany,412,412,United States
2366,645,Trappistes Rochefort 10,207,Brasserie de Rochefort,2005-12-25 11:00:00,4.85,Quadrupel (Quad),11.3,badbeer,Belgium,412,412,United States
2378,2576,Val-Dieu Triple,826,Brasserie de l'Abbaye du Val-Dieu,2006-01-25 11:00:00,4.14,Tripel,9.0,badbeer,Belgium,412,412,United States
2380,17291,Jenlain Bière De Noël,309,Brasserie Duyck,2006-05-13 10:00:00,2.58,Bière de Garde,6.8,badbeer,France,412,412,United States
2365,2646,Erdinger Weissbier Pikantus,703,Erdinger Weissbräu,2006-06-21 10:00:00,4.0,Weizenbock,7.3,badbeer,Germany,412,412,United States


In [13]:
# summerize the dictionary into a new dataframe 
users_summary = pd.DataFrame(columns=['user','total_ba_ratings','country', 'main_country_rated', 'nbr_countries_rated', 'fav_country', 'main_style_rated', 'nbr_styles_rated','fav_style','total_ratings'])
for key in users_dict:
    by_user = {
        'user': [users_dict[key]["user_name"].iloc[0]],
        'total_ba_ratings': [len(users_dict[key])],
        'country': [users_dict[key]["user_country"].iloc[0]],
        'main_country_rated':[users_dict[key]["country"].value_counts().keys()[
            users_dict[key]["country"].value_counts().argmax()]],
        'nbr_countries_rated':[users_dict[key]["country"].nunique()],
        'fav_country':[users_dict[key].sort_values(by="rating",ascending=False)["country"][:3].value_counts().keys()[
            users_dict[key].sort_values(by="rating",ascending=False)["country"][:3].value_counts().argmax()]],
        'main_style_rated': [users_dict[key]["style"].value_counts().keys()[
            users_dict[key]["style"].value_counts().argmax()]],
        'nbr_styles_rated': [users_dict[key]["style"].nunique()],
        'fav_style':[users_dict[key].sort_values(by="rating",ascending=False)["style"][:3].value_counts().keys()[
            users_dict[key].sort_values(by="rating",ascending=False)["style"][:3].value_counts().argmax()]],
        'total_ratings': [users_dict[key]["user_nbr_ratings"].iloc[0]],
    }
    users_summary = pd.concat([users_summary, pd.DataFrame(by_user)], axis=0)
    
users_summary.reset_index(drop=True, inplace=True)

In [14]:
users_summary.loc[users_summary["total_ba_ratings"]>30].head(10)

,user,total_ba_ratings,country,main_country_rated,nbr_countries_rated,fav_country,main_style_rated,nbr_styles_rated,fav_style,total_ratings
20,AlCaponeJunior,40,United States,United States,3,United States,American IPA,24,German Pilsener,1247
39,ATLbeerDog,39,United States,United States,7,United States,American IPA,25,American Double / Imperial Stout,546
41,badbeer,32,United States,United States,4,Belgium,Quadrupel (Quad),21,Saison / Farmhouse Ale,412
42,baggio,44,Sweden,Sweden,7,Belgium,American Double / Imperial IPA,24,Gueuze,376
44,Barnoloid,31,United States,United States,4,United States,American Pale Ale (APA),21,Gose,650
48,BeerBelcher,93,United States,United States,7,United States,American IPA,45,American Double / Imperial IPA,960
59,berley31,32,Canada,Canada,4,United States,American Amber / Red Ale,23,Belgian Strong Dark Ale,488
75,bluejacket74,37,United States,United States,5,Belgium,American IPA,25,Saison / Farmhouse Ale,2766
78,BMMillsy,188,United States,United States,7,United States,American IPA,54,American Double / Imperial IPA,2346
140,cpetrone84,239,United States,United States,8,Belgium,American Wild Ale,53,Gueuze,2184


This dataframe summarizes for each user the total number of ratings listed in the matched dataset, the country of the user, the main country where the beers that the user rated are brewed, the number of different countries where the beers that the user rated are brewed, the favorite country of the user, the main style of beers that the user rated, the number of different styles that the user rated, his favorite style and the total number of ratings that the user did.
With this dataframe we can do statistical and correlation analyses, e.g. correlation of the country of the user and prefered style.
NB: the matched dataset was used in this analysis, thus we don't have all the ratings of each user, however for the project we will not use the matched dataset, we will in fact use the data from ratings.txt and reviews.txt.